In [2]:
import importlib

import numpy as np

from network_models.soundsream_models_and_utils.mapping_down.ss_direct_downmapping import SS_Direct_Downmapping_Model
from network_models.soundsream_models_and_utils.encoder.ss_encoder_downmapping import EncoderDownmapping
from network_models.soundsream_models_and_utils.ss_encoded_dataset import ss_encoded_dataset_full
import torch
import gc

device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size = 8
models_dir = "/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/downmapping/0_0216/"
epochs =200
save_every = 40
start_lr = 1e-4
gc.collect()

data_set= ss_encoded_dataset_full(
    csvPath="/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/data/allEncodings_version0_12_1.pkl", device=device)

model = SS_Direct_Downmapping_Model(dropout=0.2, output=1024, start_dim=512 * 175).to(device)

In [2]:
import sys
from pathlib import Path

module_path = str(Path.cwd().parents[0] / "network_models/soundstream_models_and_utils/encoder")
if module_path not in sys.path:
    sys.path.append(module_path)
import network_models.soundsream_models_and_utils.mapping_down.ss_driect_dm_trainer as sset
importlib.reload(sset)


enc_trainer = sset.SSDirectDMTrainer(batch_size=batch_size, num_epochs=epochs, model_path=models_dir, save_model_every=save_every, lr=start_lr, dataset=data_set, device=device, model=model)
gc.collect()
#enc_trainer.train()

NameError: name 'BatchSamplerForDiffClasses' is not defined

In [3]:
np1 = data_set.encoded_dataset.encodedData[data_set.encoded_dataset.labelcolumn].to_numpy()

In [45]:
class CustomBatchSampler(object):
    def __init__(self, targets, num_samples=1):
        self.targets = targets
        self.num_samples = num_samples
        self.num_classes = len(np.unique(self.targets))

    def __iter__(self):
        for i in range(self.num_classes):
            idxs = np.where(self.targets == i)[0]
            np.random.shuffle(idxs)
            num_batches = len(idxs) // self.num_samples
            for j in range(num_batches):
                  idxs[j*self.num_samples:(j+1)*self.num_samples]

    def __len__(self):
        return len(self.targets) // self.num_samples

In [101]:
class CustomBatchSampler(object):
    def __init__(self, targets):
        self.targets = targets
        self.num_classes = len(np.unique(self.targets))
        maxLen = np.max(np.unique(targets, return_counts=True))
        # Find the indices of the samples for each class

        self.class_idxs = {}
        for i in range(self.num_classes):

            idxs = np.where(self.targets == i)[0]
            rng = np.random.default_rng(200)
            extra = rng.choice(idxs, maxLen-len(idxs))
            if len(idxs) > 0:
                self.class_idxs[i] = np.append(idxs, extra)


    def __iter__(self):
        local_indices = self.class_idxs.copy()
        rng = np.random.default_rng()


        for i in list(local_indices.keys()):
            rng.shuffle(local_indices[i])

        for i in range(len(self.class_idxs[0])):
            batch_idxs = []
            for i in list(local_indices.keys()):  # Make a copy of the keys
                idxs = local_indices[i][:1]
                batch_idxs.extend(idxs)
                local_indices[i] = np.delete(local_indices[i], np.arange(len(idxs)))
            yield batch_idxs

    def __len__(self):
        return sum([len(idxs) for idxs in self.class_idxs.values()])

    #
    # def __iter__(self):
    #     while len(self.class_idxs) > 0:
    #         batch_idxs = []
    #         for i in self.class_idxs.keys():
    #             idxs = self.class_idxs[i][:1]
    #             batch_idxs.extend(idxs)
    #             self.class_idxs[i] = np.delete(self.class_idxs[i], np.arange(len(idxs)))
    #             if len(self.class_idxs[i]) == 0:
    #                 del self.class_idxs[i]
    #             if len(batch_idxs) == self.num_classes:
    #                 break
    #         yield batch_idxs
    #
    # def __len__(self):
    #     return sum([len(idxs) for idxs in self.class_idxs.values()])

In [136]:
from torch.utils.data import DataLoader

l = [np1[i][0] for i in range(len(np1))]


dl = DataLoader(data_set, batch_sampler=CustomBatchSampler(np1))


for batch, (X, z) in enumerate(dl):
    print(np.argmax(z, axis=1))
    print(batch)
    print(X[0][0][0][170])

    if batch == 1:
        break



tensor([0, 1, 2, 3, 4, 5, 6])
0
tensor(4.6179)
tensor([0, 1, 2, 3, 4, 5, 6])
1
tensor(2.2523)


In [137]:
for batch, (X, z) in enumerate(dl):
    print(np.argmax(z, axis=1))
    print(batch)

tensor([0, 1, 2, 3, 4, 5, 6])
0
tensor([0, 1, 2, 3, 4, 5, 6])
1
tensor([0, 1, 2, 3, 4, 5, 6])
2
tensor([0, 1, 2, 3, 4, 5, 6])
3
tensor([0, 1, 2, 3, 4, 5, 6])
4
tensor([0, 1, 2, 3, 4, 5, 6])
5
tensor([0, 1, 2, 3, 4, 5, 6])
6
tensor([0, 1, 2, 3, 4, 5, 6])
7
tensor([0, 1, 2, 3, 4, 5, 6])
8
tensor([0, 1, 2, 3, 4, 5, 6])
9
tensor([0, 1, 2, 3, 4, 5, 6])
10
tensor([0, 1, 2, 3, 4, 5, 6])
11
tensor([0, 1, 2, 3, 4, 5, 6])
12
tensor([0, 1, 2, 3, 4, 5, 6])
13
tensor([0, 1, 2, 3, 4, 5, 6])
14
tensor([0, 1, 2, 3, 4, 5, 6])
15
tensor([0, 1, 2, 3, 4, 5, 6])
16
tensor([0, 1, 2, 3, 4, 5, 6])
17
tensor([0, 1, 2, 3, 4, 5, 6])
18
tensor([0, 1, 2, 3, 4, 5, 6])
19
tensor([0, 1, 2, 3, 4, 5, 6])
20
tensor([0, 1, 2, 3, 4, 5, 6])
21
tensor([0, 1, 2, 3, 4, 5, 6])
22
tensor([0, 1, 2, 3, 4, 5, 6])
23
tensor([0, 1, 2, 3, 4, 5, 6])
24
tensor([0, 1, 2, 3, 4, 5, 6])
25
tensor([0, 1, 2, 3, 4, 5, 6])
26
tensor([0, 1, 2, 3, 4, 5, 6])
27
tensor([0, 1, 2, 3, 4, 5, 6])
28
tensor([0, 1, 2, 3, 4, 5, 6])
29
tensor([0, 1, 2, 3, 

In [10]:
np.max(np.unique(np1, return_counts=True))

928

In [39]:
num_classes = len(np.unique(np1))
class_idxs = {}
maxLen = np.max(np.unique(np1, return_counts=True))


for i in range(num_classes):
    idxs = np.where(np1 == i)[0]
    rng = np.random.default_rng(200)
    extra = rng.choice(idxs, maxLen-len(idxs))
    if len(idxs) > 0:
        class_idxs[i] = np.append(idxs, extra)

In [43]:
for i in range(len(class_idxs)):
    print(len(class_idxs[i]))

928
928
928
928
928
928
928


In [34]:
rng = np.random.default_rng(200)
rng.choice(class_idxs[1], 2)

array([ 215, 4636])